# 项目3.2-神经网络压缩（知识萃取）

## 友情提示
同学们可以前往课程作业区先行动手尝试 ！！！

## 项目描述

知识萃取：用一个小model模拟出大model的行为/正确率。        

## 数据集介绍

本次使用的数据集为food-11数据集，共有11类

Bread, Dairy product, Dessert, Egg, Fried food, Meat, Noodles/Pasta, Rice, Seafood, Soup, and Vegetable/Fruit.
（面包，乳制品，甜点，鸡蛋，油炸食品，肉类，面条/意大利面，米饭，海鲜，汤，蔬菜/水果）
Training set: 9866张
Validation set: 3430张
Testing set: 3347张

**数据格式**

下载 zip 档后解压缩会有三个资料夹，分别为training、validation 以及 testing
training 以及 validation 中的照片名称格式为 [类别]_[编号].jpg，例如 3_100.jpg 即为类别 3 的照片（编号不重要）

## 项目要求

知识萃取: 让小model在学习任务的时候，藉由观察大model的行为来让自己学得更好。(直译:让小model萃取大model的知识)

## 数据准备

In [ ]:
!unzip -d work data/data58106/food-11.zip # 解压缩food-11数据集

## 环境配置/安装

无

# 简介


本项目任务是模型压缩 - Neural Network Compression。

Compression有很多种门派，在这裡我们会介绍上课出现过的其中四种，分别是:

* 知识蒸馏 Knowledge Distillation
* 网路剪枝 Network Pruning
* 用少量参数来做CNN Architecture Design
* 参数量化 Weight Quantization

在这个notebook中我们会介绍Knowledge Distillation，
而我们有提供已经学习好的大model方便大家做Knowledge Distillation。
而我们使用的小model是"Architecture Design"过的model。

* Architecute Design在同目录中的hw7_Architecture_Design.ipynb。
* 大网络的模型我们使用paddle提供的ResNet18训练的模型，路径为/home/aistudio/work/tea_net.pdparams
  * 请使用paddle提供的ResNet18，把num_classes改成11后load进去即可。(后面有范例。)

In [ ]:
import paddle
import os
import paddle.nn as nn
import paddle.optimizer as optim
import paddle.nn.functional as F
import paddle.vision.models as models

知识萃取
===
![](https://ai-studio-static-online.cdn.bcebos.com/c660e2f2598a48988c78ac90775020cdeac8654339fb4617a3023b284aac2558)

简单上来说就是让已经做得很好的大model们去告诉小model"如何"学习。
而我们如何做到这件事情呢? 就是利用大model预测的logits给小model当作标准就可以了。

## 为什么这会work?
* 例如当data不是很乾淨的时候，对一般的model来说他是个noise，只会干扰学习。透过去学习其他大model预测的logits会比较好。
* label和label之间可能有关连，这可以引导小model去学习。例如数字8可能就和6,9,0有关係。
* 弱化已经学习不错的target(?)，避免让其gradient干扰其他还没学好的task。


## 要怎么操作?
* $Loss = \alpha T^2 \times KL(\frac{\text{Teacher's Logits}}{T} || \frac{\text{Student's Logits}}{T}) + (1-\alpha)(\text{Original Loss})$


* 以下code为甚麽要对student使用log_softmax: https://github.com/peterliht/knowledge-distillation-pytorch/issues/2
* reference: [Distilling the Knowledge in a Neural Network](https://arxiv.org/abs/1503.02531)

In [ ]:
def loss_fn_kd(outputs, labels, teacher_outputs, T=20, alpha=0.5):
    # 一般的Cross Entropy
    hard_loss = F.cross_entropy(outputs, labels) * (1. - alpha)
    # 让logits的log_softmax对目标几率(teacher的logits/T后softmax)做KL Divergence。
    soft_loss = nn.KLDivLoss(reduction='batchmean')(F.log_softmax(outputs / T, axis=-1),
                                                    F.softmax(teacher_outputs / T, axis=-1)) * (alpha * T * T)
    return hard_loss + soft_loss

# 数据处理

我们的Dataset使用的是跟Hw3 - CNN同样的Dataset，因此这个区块的Augmentation / Read Image大家参考或直接抄就好。

如果有不会的话可以回去看Hw3的colab。

需要注意的是如果要自己写的话，Augment的方法最好使用我们的方法，避免输入有差异导致Teacher Net预测不好。

In [ ]:
import os
import re
import paddle
from glob import glob
from PIL import Image
import paddle.vision.transforms as transforms


class MyDataset(paddle.io.Dataset):

    def __init__(self, folderName, transform=None):
        self.transform = transform
        self.data = []
        self.label = []

        for img_path in sorted(glob(folderName + '/*.jpg')):
            try:
                # Get classIdx by parsing image path
                # class_idx = int(re.findall(re.compile(r'\d+'), img_path)[1])
                class_idx = int(os.path.split(img_path)[-1].split("_")[0])
                #print(class_idx)
            except:
                # if inference mode (there's no answer), class_idx default 0
                class_idx = 0

            image = Image.open(img_path)
            # Get File Descriptor
            image_fp = image.fp
            image.load()
            # Close File Descriptor (or it'll reach OPEN_MAX)
            image_fp.close()

            self.data.append(image)
            self.label.append(class_idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if isinstance(idx, paddle.Tensor):
            idx = idx.tolist()
        image = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, self.label[idx]


trainTransform = transforms.Compose([
    transforms.RandomCrop(256, pad_if_needed=True, padding_mode='symmetric'),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])
testTransform = transforms.Compose([
    transforms.CenterCrop(256),
    transforms.ToTensor(),
])


def get_dataloader(mode='training', batch_size=32):

    assert mode in ['training', 'testing', 'validation']

    dataset = MyDataset(
        os.path.join("/home/aistudio/work/food-11", f'{mode}'),
        transform=trainTransform if mode == 'training' else testTransform)

    dataloader = paddle.io.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=(mode == 'training'))

    return dataloader


# 预处理

我们已经提供TeacherNet的state_dict，其架构是paddle.vision提供的ResNet18。

至于StudentNet的架构则在hw7_Architecture_Design.ipynb中。

这里我们使用的Optimizer为AdamW，没有为甚么，就纯粹我想用。

In [ ]:
# get dataloader
train_dataloader = get_dataloader('training', batch_size=32)
valid_dataloader = get_dataloader('validation', batch_size=32)

In [ ]:
class StudentNet(paddle.nn.Layer):
    '''
      在这个网络里，我们会使用Depthwise & Pointwise Convolution Layer来做model。
      你会发现，将原本的Convolution Layer换成Dw & Pw后，Accuracy通常不会降很多。

      另外，取名为StudentNet是因为这个Model等会要做Knowledge Distillation。
    '''

    def __init__(self, base=16, width_mult=1):
        '''
          参数:
            base: 这个model一开始的通道数量，每过一层都会*2，直到base*16为止。
            width_mult: 为了之后的Network Pruning使用，在base*8 chs的Layer上会 * width_mult代表剪枝后的通道数量。
        '''
        super(StudentNet, self).__init__()
        multiplier = [1, 2, 4, 8, 16, 16, 16, 16]

        bandwidth = [base * m for m in multiplier]

        for i in range(3, 7):
            bandwidth[i] = int(bandwidth[i] * width_mult)

        self.cnn = nn.Sequential(
            nn.Sequential(

                nn.Conv2D(3, bandwidth[0], 3, 1, 1),
                nn.BatchNorm2D(bandwidth[0]),
                nn.ReLU6(),
                nn.MaxPool2D(2, 2, 0),
            ),
            nn.Sequential(
                
                nn.Conv2D(bandwidth[0], bandwidth[0], 3, 1, 1, groups=bandwidth[0]),
                nn.BatchNorm2D(bandwidth[0]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[0], bandwidth[1], 1),
                nn.MaxPool2D(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[1], bandwidth[1], 3, 1, 1, groups=bandwidth[1]),
                nn.BatchNorm2D(bandwidth[1]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[1], bandwidth[2], 1),
                nn.MaxPool2D(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[2], bandwidth[2], 3, 1, 1, groups=bandwidth[2]),
                nn.BatchNorm2D(bandwidth[2]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[2], bandwidth[3], 1),
                nn.MaxPool2D(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[3], bandwidth[3], 3, 1, 1, groups=bandwidth[3]),
                nn.BatchNorm2D(bandwidth[3]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[3], bandwidth[4], 1),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[4], bandwidth[4], 3, 1, 1, groups=bandwidth[4]),
                nn.BatchNorm2D(bandwidth[4]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[4], bandwidth[5], 1),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[5], bandwidth[5], 3, 1, 1, groups=bandwidth[5]),
                nn.BatchNorm2D(bandwidth[5]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[5], bandwidth[6], 1),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[6], bandwidth[6], 3, 1, 1, groups=bandwidth[6]),
                nn.BatchNorm2D(bandwidth[6]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[6], bandwidth[7], 1),
            ),

            nn.AdaptiveAvgPool2D((1, 1)),
        )
        self.fc = nn.Sequential(
            nn.Linear(bandwidth[7], 11),
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.reshape((out.shape[0], -1))
        return self.fc(out)

teacher_net = models.resnet18(pretrained=False, num_classes=11)
stu_net = StudentNet(base=16)

tea_layer_state_dict = paddle.load(
        "/home/aistudio/work/pre_tea_net.pdparams")
teacher_net.set_dict(tea_layer_state_dict)
optimizer = optim.AdamW(learning_rate=1e-3, parameters=stu_net.parameters())

# 开始训练

* 剩下的步骤与你在做Hw3 - CNN的时候一样。

## 小提醒

* paddle.no_grad是指接下来的运算或该tensor不需要算gradient。
* model.eval()与model.train()差在于Batchnorm要不要纪录，以及要不要做Dropout。

In [13]:
def run_epoch(dataloader, teacher_net, student_net, optim, update=True, alpha=0.5):
    total_num, total_hit, total_loss = 0, 0, 0
    for now_step, batch_data in enumerate(dataloader()):
        # 清空 optimizer
        optim.clear_grad()
        # 处理 input
        inputs, hard_labels = batch_data
        # 因为Teacher没有要backprop，所以我们使用paddle.no_grad
        # 告诉paddle不要暂存中间值(去做backprop)以浪费记忆体空间。
        with paddle.no_grad():
            soft_labels = teacher_net(inputs)

        if update:
            logits = student_net(inputs)
            # 使用我们之前所写的融合soft label&hard label的loss。
            # T=20是原始论文的参数设定。
            loss = loss_fn_kd(logits, hard_labels, soft_labels, 20, alpha)
            loss.backward()
            optim.clear_grad()
            optim.step()
        else:
            # 只是算validation acc的话，就开no_grad节省空间。
            with paddle.no_grad():
                logits = student_net(inputs)
                loss = loss_fn_kd(logits, hard_labels, soft_labels, 20, alpha)

        total_hit += (paddle.argmax(logits, axis=-1).numpy() == hard_labels.numpy).sum()
        #print("pre logits: ", paddle.argmax(logits, axis=-1).numpy())
        #print("hard_lables: ", hard_labels.numpy)
        #print("soft_labels: ", paddle.argmax(soft_labels, axis=-1).numpy())
        total_num += len(inputs)

        total_loss += loss.numpy()[0] * len(inputs)
    return total_loss / total_num, total_hit / total_num


# TeacherNet永远都是Eval mode.
teacher_net.eval()
now_best_acc = 0
for epoch in range(200):
    stu_net.train()
    train_loss, train_acc = run_epoch(train_dataloader, teacher_net, stu_net, optimizer, update=True)
    stu_net.eval()
    valid_loss, valid_acc = run_epoch(valid_dataloader, teacher_net, stu_net, optimizer, update=False)

    # 存下最好的model。
    if valid_acc > now_best_acc:
        now_best_acc = valid_acc
        paddle.save(stu_net.state_dict(), 'student_model.pdparams')
    print('epoch {:>3d}: train loss: {:6.4f}, acc {:6.4f} valid loss: {:6.4f}, acc {:6.4f}'.format(
        epoch, train_loss, train_acc, valid_loss, valid_acc))